# Check Number of FEL trains (single bunch) in TimePix HDF5 files

In [8]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


/home/cpassow/.conda/envs/CAMP/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['unique']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


#### Imports and utility functions

In [9]:
from src.timepix_utils import *
import src.timepix_utils as timepix_utils

timepix_utils.file_system = 'core'

### Interval of runs

In [10]:
run_interval = [526, 527] 

### List number of trains in runs

In [11]:
list_number_of_trains_vs_run(run_interval)

RunNumber: 526 | number of trains: 2619
RunNumber: 527 | number of trains: 2858


### Plot number of trains in runs

In [12]:
plot_number_of_trains_vs_run(run_interval)

### List number of trains in delay steps

In [5]:
list_number_of_trains_vs_delay(run_interval)

PermissionError: [Errno 13] Permission denied: '/home/bl1user/Desktop/erk20919/CAMP/beamtime/run_pp-delay.txt'

### List number of trains in delay steps

In [ ]:
plot_number_of_trains_vs_delay(run_interval)
